In [ ]:
# Sudoku Solver using Backtracking
# Helper function that looks is there the same number in a row or a column
# and returns False if there is same number, or returns True when the move is valid
# and there is no such a number yet
def is_valid_move(grid, row, col, num):

  for x in range(9):
      if grid[row][x] == num:
          return False

  for x in range(9):
      if grid[x][col] == num:
          return False

  corner_row = row - row % 3
  corner_col = col - col % 3
  for x in range(3):
      for y in range(3):
          if grid[corner_row + x][corner_col + y] == num:
            return False

  return True

def solve(grid, row, col):

# checking if we have reached the end of sudoku and it's solved, if not we proceed
  if col == 9:
      if row == 8:
         return True
      row += 1
      col = 0

  if grid[row][col] > 0:
      return solve(grid, row, col + 1)

  for num in range(1, 10):
      if is_valid_move(grid, row, col, num):
          grid[row][col] = num
          if solve(grid, row, col + 1):
              return True
      grid[row][col] = 0

  return False

grid = [[5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]]

if solve(grid, 0, 0):
    for i in range(9):
        for j in range(9):
            print(grid[i][j], end=" ")
        print()
else:
    print("No solution exists")


5 3 4 6 7 8 9 1 2 
6 7 2 1 9 5 3 4 8 
1 9 8 3 4 2 5 6 7 
8 5 9 7 6 1 4 2 3 
4 2 6 8 5 3 7 9 1 
7 1 3 9 2 4 8 5 6 
9 6 1 5 3 7 2 8 4 
2 8 7 4 1 9 6 3 5 
3 4 5 2 8 6 1 7 9 


In [ ]:
import numpy as np
import pandas as pd
import keras
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import *
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# Data Loading and Preprocessing
# Imports necessary libraries, loads Sudoku dataset from csv file,
# and prepares it for use in a machine learning model.
path = "drive/MyDrive/introAI/SudokuSolver/"
data = pd.read_csv(path+"sudoku.csv")
try:
    data = pd.DataFrame({"quizzes":data["puzzle"],"solutions":data["solution"]})
except:
    pass
data.head()

,quizzes,solutions
0,0043002090050090010700600430060020871900074000...,8643712593258497619712658434361925871986574322...
1,0401000501070039605200080000000000170009068008...,3461792581875239645296483719658324174729168358...
2,6001203840084590720000060050002640300700800069...,6951273841384596727248369158512647392739815469...
3,4972000001004000050000160986203000403009000000...,4972583161864397252537164986293815473759641828...
4,0059103080094030600275001000300002010008200070...,4659123781894735623275681497386452919548216372...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   quizzes    1000000 non-null  object
 1   solutions  1000000 non-null  object
dtypes: object(2)
memory usage: 15.3+ MB


In [ ]:
print("Quiz:\n",np.array(list(map(int,list(data['quizzes'][0])))).reshape(9,9))
print("Solution:\n",np.array(list(map(int,list(data['solutions'][0])))).reshape(9,9))

Quiz:
 [[0 0 4 3 0 0 2 0 9]
 [0 0 5 0 0 9 0 0 1]
 [0 7 0 0 6 0 0 4 3]
 [0 0 6 0 0 2 0 8 7]
 [1 9 0 0 0 7 4 0 0]
 [0 5 0 0 8 3 0 0 0]
 [6 0 0 0 0 0 1 0 5]
 [0 0 3 5 0 8 6 9 0]
 [0 4 2 9 1 0 3 0 0]]
Solution:
 [[8 6 4 3 7 1 2 5 9]
 [3 2 5 8 4 9 7 6 1]
 [9 7 1 2 6 5 8 4 3]
 [4 3 6 1 9 2 5 8 7]
 [1 9 8 6 5 7 4 3 2]
 [2 5 7 4 8 3 9 1 6]
 [6 8 9 7 3 4 1 2 5]
 [7 1 3 5 2 8 6 9 4]
 [5 4 2 9 1 6 3 7 8]]


In [ ]:
# A custom Data Generator class that inherits from keras.utils.Sequence
# which is used to load and preprocess data in batches during the training of the Neural network
#Utility Functions
class DataGenerator(Sequence):
    def __init__(self, df,batch_size = 16,subset = "train",shuffle = False, info={}):
        super().__init__()
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.subset = subset
        self.info = info

        self.data_path = path
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df)/self.batch_size))
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle==True:
            np.random.shuffle(self.indexes)

    def __getitem__(self,index):
        X = np.empty((self.batch_size, 9,9,1))
        y = np.empty((self.batch_size,81,1))
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        for i,f in enumerate(self.df['quizzes'].iloc[indexes]):
            self.info[index*self.batch_size+i]=f
            X[i,] = (np.array(list(map(int,list(f)))).reshape((9,9,1))/9)-0.5
        if self.subset == 'train':
            for i,f in enumerate(self.df['solutions'].iloc[indexes]):
                self.info[index*self.batch_size+i]=f
                y[i,] = np.array(list(map(int,list(f)))).reshape((81,1)) - 1
        if self.subset == 'train': return X, y
        else: return X

In [ ]:
# Model Building and Compilation
# Defines the architecture of a Convolutional Neural Network using Keras
# Adds layers, compiles the model with Adam optimizer, prints the model summary
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

adam = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 9, 9, 64)            │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 9, 9, 64)            │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 9, 9, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 9, 9, 64)            │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 9, 9, 128)           │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 10368)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 729)                 │       7,559,001 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 81, 9)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 81, 9)               │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,605,401 (29.01 MB)

 Trainable params: 7,605,145 (29.01 MB)

 Non-trainable params: 256 (1.00 KB)

In [ ]:
# Training and evaluation
# splits the data into training and validation sets
# creates data generators for each set
train_idx = int(len(data)*0.95)
data = data.sample(frac=1).reset_index(drop=True)
training_generator = DataGenerator(data.iloc[:train_idx], subset = "train", batch_size=640)
validation_generator = DataGenerator(data.iloc[train_idx:], subset = "train",  batch_size=640)

In [ ]:
training_generator.__getitem__(4)[0].shape

(640, 9, 9, 1)

In [ ]:
# callbacks for model saving and learning rate reduction
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
filepath1="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.keras"
filepath2 = "best_weights.keras"
checkpoint1 = ModelCheckpoint(filepath1, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=1,
    min_lr=1e-6
)
callbacks_list = [checkpoint1,checkpoint2,reduce_lr]

In [ ]:
# Training 1
history = model.fit(training_generator, validation_data = validation_generator, epochs = 1, verbose=1,callbacks=callbacks_list )

1484/1484 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5347 - loss: 1.2106
Epoch 1: val_accuracy improved from -inf to 0.81390, saving model to weights-improvement-01-0.81.keras

Epoch 1: val_accuracy improved from -inf to 0.81390, saving model to best_weights.keras
1484/1484 ━━━━━━━━━━━━━━━━━━━━ 1966s 1s/step - accuracy: 0.5348 - loss: 1.2103 - val_accuracy: 0.8139 - val_loss: 0.4093 - learning_rate: 0.0010


In [ ]:
model.load_weights('best_weights.keras')

In [ ]:
#testing part
test_data = data.iloc[train_idx:]
test_data = test_data.reset_index(drop=True)

test_generator = DataGenerator(test_data, subset="test", batch_size=640)
predictions = model.predict(test_generator)
predicted_grids = []
for prediction in predictions:
    predicted_grid = np.argmax(prediction, axis=1).reshape(9, 9) + 1
    predicted_grids.append(predicted_grid)

for i in range(len(predicted_grids)):
    print("Quiz:")
    print(np.array(list(map(int, list(test_data['quizzes'][i])))).reshape(9, 9)) # Now uses the reset index
    print("Predicted Solution:")
    print(predicted_grids[i])
    print("Actual Solution:")
    print(np.array(list(map(int, list(test_data['solutions'][i])))).reshape(9, 9))
    print("-" * 20)

Streaming output truncated to the last 5000 lines.
 [6 2 8 5 4 1 3 7 9]
 [4 7 9 6 3 2 5 1 8]
 [5 1 6 2 9 4 8 3 7]
 [8 3 2 7 6 5 9 4 1]
 [7 9 4 8 1 3 6 2 5]
 [2 8 3 1 5 6 7 9 4]
 [1 4 7 3 8 9 2 5 6]
 [9 6 5 4 2 7 1 8 3]]
--------------------
Quiz:
[[0 0 0 9 2 0 0 7 5]
 [0 4 0 1 0 3 2 0 0]
 [9 0 0 0 0 0 6 0 0]
 [3 0 0 0 5 7 0 0 0]
 [0 0 5 3 0 0 0 4 0]
 [0 8 6 0 4 0 9 0 0]
 [8 2 0 0 0 0 4 6 7]
 [0 0 9 7 0 0 0 0 1]
 [6 0 0 4 8 2 5 3 0]]
Predicted Solution:
[[1 6 3 9 2 4 1 7 5]
 [5 4 8 1 7 3 2 9 9]
 [9 5 2 5 7 4 6 1 4]
 [3 9 4 8 5 7 8 2 6]
 [2 9 5 3 9 8 7 4 6]
 [7 8 6 2 4 1 9 5 3]
 [8 2 3 5 3 1 4 6 7]
 [4 5 9 7 3 6 8 2 1]
 [6 7 1 4 8 2 5 3 9]]
Actual Solution:
[[1 6 8 9 2 4 3 7 5]
 [5 4 7 1 6 3 2 9 8]
 [9 3 2 8 7 5 6 1 4]
 [3 9 4 6 5 7 1 8 2]
 [2 1 5 3 9 8 7 4 6]
 [7 8 6 2 4 1 9 5 3]
 [8 2 3 5 1 9 4 6 7]
 [4 5 9 7 3 6 8 2 1]
 [6 7 1 4 8 2 5 3 9]]
--------------------
Quiz:
[[2 0 0 9 0 0 8 7 0]
 [0 0 3 0 0 0 0 1 0]
 [0 0 0 0 8 5 2 0 6]
 [8 0 2 0 0 0 3 0 0]
 [6 0 1 5 0 0 4 0 9]
 [5 0 0 7 0 8 

In [ ]:
history = model.fit(training_generator, validation_data = validation_generator, epochs = 5, verbose=1, callbacks=callbacks_list)

Epoch 1/5
1484/1484 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8180 - loss: 0.3966
Epoch 1: val_accuracy improved from 0.81390 to 0.82129, saving model to weights-improvement-01-0.82.keras

Epoch 1: val_accuracy improved from 0.81390 to 0.82129, saving model to best_weights.keras
1484/1484 ━━━━━━━━━━━━━━━━━━━━ 2006s 1s/step - accuracy: 0.8180 - loss: 0.3966 - val_accuracy: 0.8213 - val_loss: 0.3843 - learning_rate: 0.0010
Epoch 2/5
1484/1484 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8259 - loss: 0.3749
Epoch 2: val_accuracy improved from 0.82129 to 0.82594, saving model to weights-improvement-02-0.83.keras

Epoch 2: val_accuracy improved from 0.82129 to 0.82594, saving model to best_weights.keras
1484/1484 ━━━━━━━━━━━━━━━━━━━━ 1966s 1s/step - accuracy: 0.8259 - loss: 0.3749 - val_accuracy: 0.8259 - val_loss: 0.3721 - learning_rate: 0.0010
Epoch 3/5
1484/1484 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8324 - loss: 0.3599
Epoch 3: val_accuracy improved from 0.82594 to 0.82837, 

In [ ]:
#testing part
test_data = data.iloc[train_idx:]  # Or load a new test dataset
test_data = test_data.reset_index(drop=True)  # Reset the index

test_generator = DataGenerator(test_data, subset="test", batch_size=640)
predictions = model.predict(test_generator)
predicted_grids = []
for prediction in predictions:
    predicted_grid = np.argmax(prediction, axis=1).reshape(9, 9) + 1
    predicted_grids.append(predicted_grid)

for i in range(len(predicted_grids)):
    print("Quiz:")
    print(np.array(list(map(int, list(test_data['quizzes'][i])))).reshape(9, 9)) # Now uses the reset index
    print("Predicted Solution:")
    print(predicted_grids[i])
    print("Actual Solution:")
    print(np.array(list(map(int, list(test_data['solutions'][i])))).reshape(9, 9))
    print("-" * 20)

Streaming output truncated to the last 5000 lines.
 [6 2 8 5 4 1 3 7 9]
 [4 7 9 6 3 2 5 1 8]
 [5 1 6 2 9 4 8 3 7]
 [8 3 2 7 6 5 9 4 1]
 [7 9 4 8 1 3 6 2 5]
 [2 8 3 1 5 6 7 9 4]
 [1 4 7 3 8 9 2 5 6]
 [9 6 5 4 2 7 1 8 3]]
--------------------
Quiz:
[[0 0 0 9 2 0 0 7 5]
 [0 4 0 1 0 3 2 0 0]
 [9 0 0 0 0 0 6 0 0]
 [3 0 0 0 5 7 0 0 0]
 [0 0 5 3 0 0 0 4 0]
 [0 8 6 0 4 0 9 0 0]
 [8 2 0 0 0 0 4 6 7]
 [0 0 9 7 0 0 0 0 1]
 [6 0 0 4 8 2 5 3 0]]
Predicted Solution:
[[1 6 3 9 2 4 1 7 5]
 [5 4 8 1 7 3 2 9 9]
 [9 5 2 5 7 4 6 1 3]
 [3 9 4 8 5 7 1 1 6]
 [2 9 5 3 9 9 7 4 6]
 [7 8 6 2 4 1 9 5 3]
 [8 2 3 5 3 1 4 6 7]
 [4 5 9 7 3 6 8 8 1]
 [6 7 1 4 8 2 5 3 9]]
Actual Solution:
[[1 6 8 9 2 4 3 7 5]
 [5 4 7 1 6 3 2 9 8]
 [9 3 2 8 7 5 6 1 4]
 [3 9 4 6 5 7 1 8 2]
 [2 1 5 3 9 8 7 4 6]
 [7 8 6 2 4 1 9 5 3]
 [8 2 3 5 1 9 4 6 7]
 [4 5 9 7 3 6 8 2 1]
 [6 7 1 4 8 2 5 3 9]]
--------------------
Quiz:
[[2 0 0 9 0 0 8 7 0]
 [0 0 3 0 0 0 0 1 0]
 [0 0 0 0 8 5 2 0 6]
 [8 0 2 0 0 0 3 0 0]
 [6 0 1 5 0 0 4 0 9]
 [5 0 0 7 0 8 